In [1]:
import tensorflow as tf
import numpy as np

print(tf.version.VERSION)

2.9.1


In [2]:
text1 = open('novel\天龍八部.txt',encoding='utf-8').read()
text2= open('novel\射雕英雄傳.txt', encoding='utf-8').read()
text3 =open('novel\笑傲江湖.txt',encoding='utf-8').read()
text4 =open('novel\倚天屠龍記.txt',encoding='utf-8').read()
text5 = open('novel\白馬嘯西風.txt',encoding='utf-8').read()
text6= open('novel\俠客行.txt', encoding='utf-8').read()
text7 =open('novel\雪山飛狐.txt',encoding='utf-8').read()
text8 =open('novel\碧血劍.txt',encoding='utf-8').read()
text= text1 + text2 +text3 +text4 +text5 + text6+ text7 + text8

In [3]:
w = len(set(text))
tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=w,char_level=True,filters='')
tokenizer.fit_on_texts(text)

In [4]:
EMBEDDING_DIM = 512
RNN_UNITS = 1024
BATCH_SIZE = 1
# w =4880
# 專門用來做生成的模型容器
infer_model = tf.keras.Sequential()

# 詞嵌入層
infer_model.add(
    tf.keras.layers.Embedding(
        input_dim=w, 
        output_dim=EMBEDDING_DIM,
        batch_input_shape=[BATCH_SIZE, None]
        ))

# LSTM 層
infer_model.add(tf.keras.layers.LSTM(units=RNN_UNITS, return_sequences=True, stateful=True))

# 全連接層
infer_model.add(tf.keras.layers.Dense(w))

infer_model.build(tf.TensorShape([1, None]))
infer_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (1, None, 512)            2692096   
                                                                 
 lstm (LSTM)                 (1, None, 1024)           6295552   
                                                                 
 dense (Dense)               (1, None, 5258)           5389450   
                                                                 
Total params: 14,377,098
Trainable params: 14,377,098
Non-trainable params: 0
_________________________________________________________________


In [5]:
def generate_text(model, start_string,words_number,temperature): 
  # Number of characters to generate
  num_generate = words_number #字數調整，前端更改

  input_eval = tokenizer.texts_to_sequences([start_string])[0]
  input_eval = tf.expand_dims(input_eval, 0)
  # Empty string to store our results
  text_generated = []
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions,
      num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(tokenizer.index_word[predicted_id])

  return (start_string + ''.join(text_generated))

In [6]:
h5_path='myModel_8books_10Words_200Epochs.h5'
infer_model.load_weights(h5_path)
infer_model.build(
    tf.TensorShape([1, None]))

In [7]:
# 呼叫預測函數
'''
起始示範語句：
1.少俠皮皮蝦一登上光明頂
2.掌門人皮皮蝦手一揮掌勁一使，
3.掌門人皮皮蝦雙手一揮掌勁一使，
'''
results = generate_text(infer_model, start_string="掌門人皮皮蝦右手一揮掌勁一使，",words_number=100,temperature=0.6)
print(results)

掌門人皮皮蝦右手一揮掌勁一使，更是虛招，便即明白是他們打扮之德，何以連擬瓜邪派令狐沖意說的。是以大叉上山，便是為我們下毒之後，當真是一個人，圖中一腕尤叩山睜，自是大為震驚。
　　貝海石道：“姓袁的，你說要報仇，還不是一題？”
　　


In [8]:
# print(generate_text(infer_model, start_string="少俠盧瑞山身子一躍"))

In [9]:
import pandas as pd
import time
import flask
from flask import jsonify, request
from flask_cors import CORS

app = flask.Flask(__name__)
#app.config["DEBUG"] = False
app.config["JSON_AS_ASCII"] = False
cors = CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/', methods=['GET'])
def home():
    return "<h1>Hello Flask!</h1>"

@app.route('/text_gen', methods=['GET'])
def text_gen():
    if 'start_string' in request.args:
        start_string = request.args['start_string']
        words_number=int(request.args.get('words_number'))
        temperature=float(request.args.get('temperature'))
        print(start_string,words_number,temperature)
    else:
        return "Error: No city_name provided. Please specify a city_name."
    
    results = []    
    start_time = time.time()    
    results = generate_text(infer_model, start_string=start_string,words_number=words_number,temperature=temperature) 
    #print(results)
    end_time = time.time()
    print('推論花了：',end_time - start_time,'秒的時間')
    return results

app.run(host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.161.48:5000 (Press CTRL+C to quit)
